In [7]:
import telebot
from telebot import types
import requests
import config as cfg
from Get_weather import get_weather
from Get_weather import get_coordinate
from Matching_smile import matchig_weather
from Matching_smile import mathing_wind

In [10]:
bot = telebot.TeleBot(cfg.TG_TOKEN)
'''
get_weather - запрос погоды по координатам
get_coordinate - получение координат города
matchig_weather - получение смайла погоды, в зависимости от иконки
mathing_wind - получение направления ветра в зависимости от градуса.
'''

def create_message(message, latitude, longitude):
    lang = message.from_user.language_code
    description,temp,wind,city,icon, wind_deg = get_weather(latitude, longitude, lang)
    icon_smile = matchig_weather(icon)
    icon_wind = mathing_wind(wind_deg)
    
    return f'<u>{city}</u>:\n\n🌡 {temp}°C\n{icon_smile} <i>{description}</i>\nВетер: 🌬 {wind}м/с {icon_wind}'
    

@bot.message_handler(commands=['start'])
def handler_start(message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    button = types.KeyboardButton('Узнать погоду', request_location = True)
    
    keyboard.add(button)
    bot.reply_to(message, 'Нажми на кнопку и узнай погоду', reply_markup=keyboard)
    
    
@bot.message_handler(content_types=['location'])
def location_message(message):
    latitude = message.location.latitude
    longitude = message.location.longitude

    mess = create_message(message,latitude, longitude)
    bot.send_message(message.chat.id, mess, parse_mode="HTML")


    
@bot.message_handler(content_types=['text'])
def location_message(message):
    data = get_coordinate(message.text)
    if data: 
        latitude, longitude = data
        mess = create_message(message,latitude, longitude)
        bot.send_message(message.chat.id, mess, parse_mode="HTML")   
    else: 
       bot.send_message(message.chat.id, f'Город не удалось найти') 
  
    
    

bot.polling()

ReadTimeout: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)